In [2]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

import sklearn
import csv
import pandas as pd
import pymatgen as mg
import random
import os
from bisect import bisect_left   
from sklearn import preprocessing
from sklearn.datasets.base import Bunch
from sklearn import svm

#from sklearn import datasets, svm
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

style.use("ggplot")

/home/willcai/anaconda3/envs/my_pymatgen/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#feature dictionary
elements_data = pd.read_csv('features_csv.csv')
elements = elements_data['Symbol']
atomic_numbers = elements_data['Number']
feature_names = elements_data.columns.values[2:]

print('features: {}'.format(feature_names))

features_dict = dict()

for item in feature_names:
    features_dict[item] = dict(zip(elements, elements_data[item]))


print(features_dict)


features: ['Density' 'Molar volume' 'BCCefflatcnt' 'NfUnfilled' 'NpUnfilled'
 'BCCenergy_pa' 'Thermal conductivity' 'IsMetal' 'MendeleevNumber'
 'BCCvolume_pa' 'IsFBlock' 'BCCfermi' 'Velocity of sound' 'IsNonmetal'
 'CovalentRadius' 'GSefflatcnt' 'GSvolume_pa' 'BCCvolume_padiff'
 'SpaceGroupNumber' 'GSestFCClatcnt' 'NdUnfilled' 'GSmagmom' 'BoilingT'
 'Row' 'GSestBCClatcnt' 'Electronegativity' 'BulkModulus' 'MeltingT'
 'NsValence' 'Van der waals radius' 'BCCbandgap' 'NUnfilled' 'HeatFusion'
 'IsMetalloid' 'HeatCapacityMolar' 'Column' 'NdValence' 'BCCenergydiff'
 'Liquid range' 'IsDBlock' 'NfValence' 'NpValence' 'AtomicWeight'
 'GSenergy_pa' 'Polarizability' 'FirstIonizationEnergy' 'NsUnfilled' 'phi'
 'BCCmagmom' 'GSbandgap' 'ICSDVolume' 'AtomicVolume' 'IsAlkali' 'NValance'
 'HeatCapacityMass']
{'Density': {'H': 0.0899, 'Li': 535.0, 'Be': 1848.0, 'B': 2460.0, 'C': 2260.0, 'N': 1.251, 'O': 1.429, 'F': 1.696, 'Na': 968.0, 'Mg': 1738.0, 'Al': 2700.0, 'Si': 2330.0, 'P': 1823.0, 'S': 1960.0, 

In [4]:
elements_data.head(85)

,Symbol,Number,Density,Molar volume,BCCefflatcnt,NfUnfilled,NpUnfilled,BCCenergy_pa,Thermal conductivity,IsMetal,...,FirstIonizationEnergy,NsUnfilled,phi,BCCmagmom,GSbandgap,ICSDVolume,AtomicVolume,IsAlkali,NValance,HeatCapacityMass
0,H,1,0.0899,11.42,3.589268,0,0,-2.135811,0.1805,0,...,13.598443,1,5.20,0.000000,7.853,32.02,18618.051940,0,1,14.304
1,Li,3,535.0000,13.02,6.416364,0,0,-1.865535,85.0000,1,...,5.391719,1,2.85,0.000000,0.000,21.63,21.544058,1,1,3.582
2,Be,4,1848.0000,4.85,4.997332,0,0,-3.655272,190.0000,1,...,9.322700,0,5.05,0.000000,0.000,8.11,8.098176,1,2,1.825
3,B,5,2460.0000,4.39,4.606670,0,5,-4.966431,27.0000,0,...,8.298020,0,5.30,0.000000,1.524,7.75,7.297767,0,3,1.026
4,C,6,2260.0000,5.29,4.726663,0,4,-4.871762,140.0000,0,...,11.260300,0,6.24,0.000000,4.496,5.67,8.825090,0,4,0.709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Th,90,11724.0000,19.80,8.078396,0,0,-7.266547,54.0000,1,...,6.306700,0,3.30,0.000000,0.000,32.86,32.865683,0,4,0.118
81,Pa,91,15370.0000,15.18,7.324581,12,0,-9.317631,47.0000,1,...,5.890000,0,0.00,0.000000,0.000,24.94,24.961161,0,5,0.000
82,U,92,19050.0000,12.49,6.866024,11,0,-11.017414,27.0000,1,...,6.194100,0,3.90,0.000000,0.000,21.18,20.748847,0,6,0.116
83,Np,93,20450.0000,11.59,6.585459,10,0,-12.498582,6.0000,1,...,6.265700,0,0.00,0.000091,0.000,19.22,19.244839,0,7,0.000


In [5]:
#code for making features
def get_fractions(formulas):
	N = len(formulas)
	fractions = np.zeros((N, 2)) #??? what is you have more than 2 elements in a compound
	for i, formula in enumerate(formulas):
		comp = mg.core.Composition(formula)
		species = comp.elements
		for j in range(2):
			fractions[i, j] = comp.get_atomic_fraction(species[j])
	return fractions





def get_mean_features(formulas, feature):
	# print feature
	N = len(formulas)
	feature_values = np.zeros(N)
	for i, formula in enumerate(formulas):
		comp = mg.core.Composition(formula)
		k = len(mg.core.Composition(formula))
		species = list(map(str,comp.elements)) #turn the double quote into single quote
		for j in range(k):
			feature_values[i] += features_dict[feature][species[j]] * comp.get_atomic_fraction(species[j])
	return feature_values


def get_product_features(formulas, feature):
	N = len(formulas)
	feature_values = np.ones(N)  # need ones, not zeros
	for i, formula in enumerate(formulas):
		comp = mg.core.Composition(formula)
		k = len(mg.core.Composition(formula))
		species = list(map(str, comp.elements))
#		species = comp.elements (can't go this code)
		for j in range(k):
			feature_values[i] *= features_dict[feature][species[j]] * comp.get_atomic_fraction(
				species[j])
	return feature_values


def get_X_from_data(raw_formulas, feature_list=feature_names):
	formulas=raw_formulas
	# formulas=[]
	# for formula in raw_formulas:
	# 	inc=True
	# 	for e in exclude:
	# 		if e in formula:
	# 			inc=False
	# 	if inc:
	# 		formulas.append(formula)
	X = get_fractions(formulas)
	# X=np.random.randint(2, size=len(data))
	for item in feature_list:
		featureValues_mean = get_mean_features(formulas, item)
		featureValues_prod = get_product_features(formulas, item)
		# featureValues_diff = get_diff_features(data, item)
		if len(np.shape(X)) == 1:
			# X=featureValues
			np.column_stack((featureValues_mean, featureValues_prod))
		else:
			X = np.column_stack((X, featureValues_mean, featureValues_prod))
	return X

#shuffles dataset
def format_dataset(X, y, size = -1):
    random.seed(a=5)
    length = np.size(y)
    indices = np.arange(length)
    np.random.shuffle(indices)
    new_X = []
    new_y = []
    for x in indices:
        new_X.append(X[x])
        new_y.append(y[x])
    if size == -1:
        return Bunch(data=new_X, target=new_y)
    return Bunch(data=new_X[0:size], target=new_y[0:size])

#pos, neg are lists
def get_labels(pos, neg):
    y = []
    for i in range(len(pos)):
        y.append(1)
    for i in range(len(neg)):
        y.append(0)
    return np.array(y)


#returns information on success of model
def accuracy(pred, test):
    counter = 0
    true_positive = 0
    false_positive = 0
    false_negative = 0
    true_negative = 0
    temp = 0
    for i in range(0,np.size(pred)):
        if pred[i] < 0.5:
            temp = 0
        else:
            temp = 1
        if test[i] == temp:
            counter += 1
        if test[i] == 1 and temp == 1:
            true_positive += 1
        if test[i] == 0 and temp == 1:
            false_positive += 1
        if test[i] == 1 and temp == 0:
            false_negative += 1
        if test[i] == 0 and temp == 0:
            true_negative += 1
    return [counter, np.size(pred), true_positive, false_positive, false_negative, true_negative]

In [6]:
#get data
with open('random_in_icsd.txt', 'r') as f:
    pos = f.readlines()
    pos = [x.strip() for x in pos]
    print(len(pos))
    #10899
with open('random_no_icsd.txt', 'r') as f:
    neg = f.readlines()
    neg = [x.strip() for x in neg]
    print(len(neg))
    #2458351

10899
2458351


In [7]:
#featurize the positive examples
#make features from formulas_list
random.seed(a=5)
# num_examples=100

#computationally intensive
num_examples = 10899

# do some samplings
# pos_sample=random.sample(pos, num_examples)

#use the whole positive population
pos_sample = pos
X_pos = get_X_from_data(pos_sample)

print(np.shape(X_pos))

(10899, 112)


In [8]:
#featurize negative examples, set here to be the same length as the number of positive examples
random.seed(a=5)
neg_sample=random.sample(neg, num_examples)
X_neg=get_X_from_data(neg_sample)

#concatenate the features together and scale
X = np.concatenate((X_pos, X_neg), axis = 0)
std_scale=preprocessing.StandardScaler().fit(X)
X = std_scale.transform(X)

#get output label
y = []

#Test 1: positive = ICSD, negative = random
#Test 2: positive = random same format, negative = random

y=get_labels(pos_sample,neg_sample)

print(np.shape(X))

(21798, 112)


In [9]:
#85% for traning and 15% for testing
train = 9264
syn = format_dataset(X, y)

#Split the data into training/testing sets.
portion = train
X_train = syn.data[:portion]

X_test = syn.data[portion:]

# Split the targets into training/testing sets
y_train = syn.target[:portion]
y_test = syn.target[portion:]
#print(y_train)

# Create regression object
#lin_reg = linear_model.LinearRegression()
#log_reg = linear_model.LogisticRegression()
#svm_lin = svm.LinearSVC()
#las_reg = linear_model.Lasso(alpha=0.1)
#rid_reg = Ridge(alpha=1.0)
#nai_bay = GaussianNB()
#ran_for = RandomForestClassifier(n_estimators=100)
#dec_tre = tree.DecisionTreeClassifier()
clf = svm.SVC(kernel='rbf', C = 1.0, cache_size = 20000)

# Train the model using the training sets
#lin_reg.fit(X_train, y_train)
#log_reg.fit(X_train, y_train)
#svm_lin.fit(X_train, y_train)
#las_reg.fit(X_train, y_train)
#rid_reg.fit(X_train, y_train)
#nai_bay.fit(X_train, y_train)
#ran_for.fit(X_train, y_train)
#dec_tre.fit(X_train, y_train)

# Make predictions using the testing set
#y_pred_lin = lin_reg.predict(X_test)


#calculate linear regression accuracy
#curr_accuracy = accuracy(y_pred_svm_li, y_test)[0] / accuracy(y_pred_svm_li, y_test)[1]
#print(curr_accuracy)

print(len(X_train))
print(np.shape(X_test))

print(np.shape(y_train))
print(np.shape(y_test))



9264
(12534, 112)
(9264,)
(12534,)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
print(X_train[500])

[-0.71152481 -0.71485086  0.42131866 -0.22447358 -1.25289251 -0.68960105
 -0.97679141 -0.89489032 -0.64287828 -0.03714069  0.10568052 -0.1264372
 -0.52226834 -0.35504477  0.56672099 -0.1284684   0.96212521  1.10969546
  0.44884416  0.64427828 -0.9378347  -0.55488948 -0.8554187  -0.080463
  0.37614465  0.55720493  0.53773001  0.59332152 -0.70656239 -0.0710322
 -0.44258434 -0.49893097 -1.24537021 -1.08399546 -1.10601019 -0.63224582
  1.01426781  0.08145193  1.12986391  1.38551037 -1.24537021 -1.08399546
 -0.32134166 -0.12781401 -0.36846542 -0.0149008   0.34890714  0.09667438
 -0.32405481 -0.64159602 -1.24537021 -1.08399546  0.13982262 -0.02110067
  1.1847749   0.83575917  0.43832686  0.06575059 -3.07050786 -2.1353381
 -0.90400927 -0.58236688  0.          0.         -0.70276326 -0.40906283
 -0.15664286 -0.18804461 -0.61606027 -0.01352055  0.02432983  0.02043382
  0.40603695  0.53633374  0.83952622 -0.31067277 -0.67017438 -0.09152496
  0.18742057 -0.07316527  1.55183985 -0.15846968 -0.9173

In [291]:
clf.fit(X_train, y_train)

SVC(cache_size=20000)

In [292]:
y_pred_svm_rbf = clf.predict(X_test)

In [293]:
curr_accuracy = accuracy(y_pred_svm_rbf, y_test)[0] / accuracy(y_pred_svm_rbf, y_test)[1]
print(curr_accuracy)

0.8625339077708633


In [294]:
# run for 100 times and calculate the average of the lin_reg_accuracy
count = 0
total_accuracy = 0
for i in range(50):
    count += 1
    train = 9264
    syn = format_dataset(X, y)

    portion = train
    X_train = syn.data[:portion]
    X_test = syn.data[portion:]

    # Split the targets into training/testing sets
    y_train = syn.target[:portion]
    y_test = syn.target[portion:]
    
    # Create regression object
    clf = svm.SVC(kernel='rbf', C = 1.0, cache_size = 20000)

    # Train the model using the training sets
    clf.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred_svm_rbf = clf.predict(X_test)

    #calculate linear regression accuracy
    total_accuracy += accuracy(y_pred_svm_rbf, y_test)[0] / accuracy(y_pred_svm_rbf, y_test)[1]

avg_curr_accuracy = total_accuracy / count
print(avg_curr_accuracy)

0.8649561193553538
